# ANN Churn Prediction

## Importing dependencies

In [15]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

## Importing dataset

In [16]:
df = pd.read_csv("dataset/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocessing

- RowNumber, CustomerId, Surname are non essential features

In [17]:
df.drop(["RowNumber", "CustomerId", "Surname"],axis=1, inplace=True)

In [18]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

- Geography & Gender are categorical variable, covert it into numerical variable. 

In [20]:
label_encoder_gender = LabelEncoder()

df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head() 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [21]:
from sklearn.preprocessing import OneHotEncoder
encoder_geography = OneHotEncoder(sparse_output=False)

onehot_encoded_geography = encoder_geography.fit_transform(df[['Geography']])
print(onehot_encoded_geography)
print(type(onehot_encoded_geography))

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
<class 'numpy.ndarray'>


In [22]:
print(encoder_geography.get_feature_names_out(['Geography']))

['Geography_France' 'Geography_Germany' 'Geography_Spain']


In [23]:
geo_df = pd.DataFrame(onehot_encoded_geography, columns=encoder_geography.get_feature_names_out(['Geography']))
print(geo_df)

      Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]


In [24]:
df_final = pd.concat([df.drop(['Geography'], axis=1), geo_df], axis=1)

df_final.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [25]:
temp = df_final['Exited']
df_final = pd.concat([df_final.drop(['Exited'], axis=1), temp], axis=1)
df_final.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0,1
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0,0


### Saving encoders 

In [26]:
with open('models/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file) 
    
with open('models/one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(encoder_geography, file)

## Independent & Dependent Feature splitting

In [48]:
X = df_final.iloc[:,:-1]
y = df_final.iloc[:,-1]

## Train test split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

In [54]:
print(f"X_train: ${X_train.shape}")
print(f"X_test: ${X_test.shape}")
print(f"y_train: ${y_train.shape}")
print(f"y_test: ${y_test.shape}")

X_train: $(8000, 12)
X_test: $(2000, 12)
y_train: $(8000,)
y_test: $(2000,)


## Scaling Features

In [55]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
with open('models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# ANN Implementation

In [70]:
import tensorflow as tf


In [71]:
print(tf.__version__)

2.18.0


In [75]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
import datetime

## Architecture

In [76]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # Hidden Layer 1
    Dense(32, activation='relu'), # Hidden Layer 2
    Dense(1, activation='sigmoid') # Output layer
])   


C:\Users\Prateek Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [77]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

### Compiling the model

In [78]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [79]:
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

### Setting up Tensorboard

In [91]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  

In [92]:
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

### Setting up Early Stopping

In [93]:
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


### Training model

In [94]:
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[early_stopping_callbacks, tensorflow_callbacks] 
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7993 - loss: 0.4967 - val_accuracy: 0.7965 - val_loss: 0.5054
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7993 - loss: 0.5007 - val_accuracy: 0.7965 - val_loss: 0.5054
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7887 - loss: 0.5153 - val_accuracy: 0.7965 - val_loss: 0.5063
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8021 - loss: 0.4965 - val_accuracy: 0.7965 - val_loss: 0.5065
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8007 - loss: 0.4986 - val_accuracy: 0.7965 - val_loss: 0.5053
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7976 - loss: 0.5037 - val_accuracy: 0.7965 - val_loss: 0.5057
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7935 - loss: 0.5085 - val_accuracy: 0.7965 - val_loss: 0.5053
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7856 - loss: 0.5200 - val_accu

In [95]:
model.save("models/model.keras")

### Load TensorBorad Extension

In [96]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [97]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 10272), started 0:05:46 ago. (Use '!kill 10272' to kill it.)